### Getting the files & import statements ###

In [4]:
import pandas as pd
import numpy as np
import os

In [10]:
# Get participant number 
participant_number = input("Enter participant number: ")

base_path = "/Users/alina/Downloads/Thesis/raw/"

# Folder path for the participant
folder_path = os.path.join(base_path, f"Participant {participant_number}")

# Get recording number 
recording_number = input("Enter recording number: ")

# Recording folder path
recording_path = os.path.join(folder_path, f"P{participant_number}_{recording_number}")

# Get files
world_timestamps_file = os.path.join(recording_path, "world_timestamps.csv")
detections_file = os.path.join(recording_path, f"detections_{participant_number}_{recording_number}.csv")
fixations_file = os.path.join(recording_path, "fixations.csv")
face_file = os.path.join(recording_path, "fixations_on_face.csv")

# Load as dfs
world_timestamps_df = pd.read_csv(world_timestamps_file) #contains timestamps for each video frame
detections_df = pd.read_csv(detections_file) #contains detections from YOLO
fixations_df = pd.read_csv(fixations_file) #fixations data
face_df = pd.read_csv(face_file) #fixations on face data from FaceMapper

### Matching video frames and timestamps

In [11]:
## function to see if the gaze is inside the bounding box of an object
def is_gaze_in_bbox(gaze_x, gaze_y, x_min, y_min, x_max, y_max):
    return x_min <= gaze_x <= x_max and y_min <= gaze_y <= y_max

## function to prioritise the smallest object fixated
def filter_objects_by_area(group):
    group = group.copy()
    group["area"] = (group["X_max"] - group["X_min"]) * (group["Y_max"] - group["Y_min"])

    # Find the minimum area
    min_area = group["area"].min()
    
    # Get all objects with the minimum area
    smallest_objects = group[group["area"] == min_area]
    
    if len(smallest_objects) == 1:
        return smallest_objects
    else:
        person_objects = smallest_objects[smallest_objects["Class"] == "person"]
        if not person_objects.empty:
            # Return only one person object (the first one)
            return person_objects.iloc[[0]]

    # If no person among smallest, return just one of the smallest objects
    return smallest_objects.iloc[[0]]

In [12]:
## world_timestamps.csv file from pupil cloud represents timestamps for each video frame
# merge_asof is like a nearest match join for ordered time-series data

# renaming so all timestamp columns are named the same to merge the dfs
fixations_df["timestamp [ns]"] = fixations_df["start timestamp [ns]"]

## making a video_df which contains matching frames and ts
ts = world_timestamps_df['timestamp [ns]']

video_df = pd.DataFrame({
        "Frame": np.arange(len(world_timestamps_df)),
        "timestamp [ns]": ts,
    })

# Make sure everything is sorted by timestamp so merge_asof works properly
fixations_df = fixations_df.sort_values("start timestamp [ns]")
world_timestamps_df = world_timestamps_df.sort_values("timestamp [ns]")

# Merge-asof to assign the closest frame_id to each fixation
fixations_with_frame = pd.merge_asof(
    fixations_df,
    video_df,
    on="timestamp [ns]",
    direction="nearest",  # to match the closest timestamp from video_df, whether it's before or after the fixation timestamp
) 

# merging to assign fixations to frames
fixations_with_detections = pd.merge(
    fixations_with_frame,
    detections_df,
    on="Frame",
    how="left" # keep all the fixations, even if a frame has no detection => nan
)

# calculate in_bbox 
fixations_with_detections["in_bbox"] = fixations_with_detections.apply(
    lambda row: is_gaze_in_bbox(
        row["fixation x [px]"], row["fixation y [px]"],
        row["X_min"], row["Y_min"], row["X_max"], row["Y_max"]
    ),
    axis=1
)

# filter only fixations that match at least one bounding box
in_bbox_fixations = fixations_with_detections[fixations_with_detections["in_bbox"] == True]

#  group by fixation ID and drop "person" if there is another class in same fixation group
filtered_fixations = in_bbox_fixations.groupby(
    "Frame", group_keys=False # grouping on frame as i need fixation id column later
).apply(filter_objects_by_area, include_groups=False)

fixations_on_persons = filtered_fixations.loc[filtered_fixations["Class"] == "person"]
fixations_on_persons.head()

,section id,recording id,fixation id,start timestamp [ns],end timestamp [ns],duration [ms],fixation x [px],fixation y [px],azimuth [deg],elevation [deg],timestamp [ns],X_min,Y_min,X_max,Y_max,Confidence,Class,in_bbox,area
3,4494865b-9b8f-42f2-b23e-c7b0ff1437fd,e124d98d-2438-4966-9da5-98817030b6b1,4,1744712357171728890,1744712357311852890,140,1023.533,1193.859,15.238139,-37.356346,1744712357171728890,770.859680,10.406494,1596.113892,1196.820679,0.917327,person,True,9.790933e+05
4,4494865b-9b8f-42f2-b23e-c7b0ff1437fd,e124d98d-2438-4966-9da5-98817030b6b1,5,1744712357456977890,1744712357622101890,165,973.939,1185.037,11.440663,-36.932289,1744712357456977890,710.183838,0.000000,1598.204224,1197.362183,0.930763,person,True,1.063282e+06
22,4494865b-9b8f-42f2-b23e-c7b0ff1437fd,e124d98d-2438-4966-9da5-98817030b6b1,12,1744712359463847890,1744712359618972890,155,1264.344,1111.778,31.812326,-31.021794,1744712359463847890,753.040466,7.975998,1599.849243,1200.000000,0.748790,person,True,1.009416e+06
25,4494865b-9b8f-42f2-b23e-c7b0ff1437fd,e124d98d-2438-4966-9da5-98817030b6b1,13,1744712359648972890,1744712359944349890,295,1275.214,1110.224,32.567667,-30.861358,1744712359648972890,697.059448,35.524979,1596.056641,1200.000000,0.458491,person,True,1.046860e+06
29,4494865b-9b8f-42f2-b23e-c7b0ff1437fd,e124d98d-2438-4966-9da5-98817030b6b1,14,1744712359969346890,1744712360229600890,260,1309.005,1111.482,35.012704,-30.725126,1744712359969346890,648.540161,37.090912,1595.309082,1199.440186,0.768677,person,True,1.100476e+06


### Creating a new file with categorised fixations

In [13]:
# filtering to only fixations on bodies
fixations_on_persons = fixations_on_persons[fixations_on_persons["in_bbox"] == True]

# initialising a df for all types of fixations
categorized_fixations_all = []

# using FaceMapper file to get all the basic fixations information
for _, row in face_df.iterrows():
    timestamp = row['start timestamp [ns]']
    fixation_id = row['fixation id']
    fix_duration = (row['end timestamp [ns]'] - row['start timestamp [ns]'])/1000000     #converting to ms
    fixation_type = "background" #make default type

    # face fixations
    if row.get('fixation on face') == True:
        fixation_type = "face"

    # body fixations (only if not already labeled as face)
    elif not fixations_on_persons[fixations_on_persons['timestamp [ns]'] == timestamp].empty:
        fixation_type = "body"
        matching_row = fixations_on_persons[fixations_on_persons['timestamp [ns]'] == timestamp].iloc[0]
        fixation_id = matching_row['fixation id']
        fix_duration = matching_row['duration [ms]']

    categorized_fixations_all.append({
        'timestamp [ns]': timestamp,
        'type': fixation_type,
        'fixation id': fixation_id,
        'duration [ms]': fix_duration
    })

# saving a file with fixations categorised by face, body and background
all_categorized_fixations_df = pd.DataFrame(categorized_fixations_all)
saving_path = os.path.join(recording_path, f"fixations_on_everything_{participant_number}_{recording_number}.csv")
all_categorized_fixations_df.to_csv(saving_path, index=False)

all_categorized_fixations_df.head()

,timestamp [ns],type,fixation id,duration [ms]
0,1744712352692613890,background,1,140.125
1,1744712353112987890,background,2,305.250
2,1744712353478250890,background,3,3523.353
3,1744712357171728890,body,4,140.000
4,1744712357456977890,body,5,165.000
